<a href="https://colab.research.google.com/github/tendiek/classifierpredict/blob/master/AC_Predict_Starter_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced Classification Predict

© Explore Data Science Academy

## Honour Code

I **YOUR NAME**, confirm - by submitting this document - that the solutions in this notebook are a result of my own work and that I abide by the EDSA honour code (https://drive.google.com/file/d/1QDCjGZJ8-FmJE3bZdIQNwnJyQKPhHZBn/view?usp=sharing).

Non-compliance with the honour code constitutes a material breach of contract.

### Import Libraries and Read In the Data

Do not modify or remove any of the code in these cells.

In [432]:
from IPython.display import Image
from IPython.display import IFrame
from IPython import display
import pandas as pd
import numpy as np

import nltk
import sklearn
import imblearn
import csv
%matplotlib inline

from nltk import TreebankWordTokenizer, SnowballStemmer
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import  word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from imblearn.metrics import classification_report_imbalanced

from sklearn.preprocessing import LabelBinarizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors  import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split

import xgboost as xgb
from xgboost import XGBClassifier

from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse.csr import csr_matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import VarianceThreshold

from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

import string
import urllib
import math
import re

nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
train  = pd.read_csv('train.csv')
test = pd.read_csv('test.csv') # no labels

train.head()

,sentiment,message,tweetid
0,1,PolySciMajor EPA chief doesn't think carbon di...,625221
1,1,It's not like we lack evidence of anthropogeni...,126103
2,2,RT @RawStory: Researchers say we have three ye...,698562
3,1,#TodayinMaker# WIRED : 2016 was a pivotal year...,573736
4,1,"RT @SoyNovioDeTodas: It's 2016, and a racist, ...",466954


In [ ]:
test.head()

,message,tweetid
0,Europe will now be looking to China to make su...,169760
1,Combine this with the polling of staffers re c...,35326
2,"The scary, unimpeachable evidence that climate...",224985
3,@Karoli @morgfair @OsborneInk @dailykos \nPuti...,476263
4,RT @FakeWillMoore: 'Female orgasms cause globa...,872928


## Overview

Refer to this diagram to guide you while you build your model. Some of the steps will be fleshed out in this template.

![Overview](process_overview_final.jpg)

## Basic preprocessing

Here is a template you may use for your initial base model. 

### Removing URL's
Write a function that removes URL's from a single tweet. 

**Function input:**
- A single string object (tweet) 

**Function output:**
- The tweet with URL's removed as a single object

In [ ]:
# define function that removes URL from single tweet
def remove_urls(text):
    # YOUR CODE HERE 
    processed_text = re.sub(r"http\S+", "", text)
    return processed_text

**Original object:**

In [ ]:
tweet = train['message'][2]
tweet

'RT @RawStory: Researchers say we have three years to act on climate change before it’s too late https://t.co/WdT0KdUr2f https://t.co/Z0ANPT…'

In [ ]:
remove_urls(tweet)

'RT @RawStory: Researchers say we have three years to act on climate change before it’s too late  '

**Expected output:**
```python
remove_urls(tweet)

>>'RT @RawStory: Researchers say we have three years to act on climate change before it’s too late  '
```

### Convert to lowercase

Write a function that converts a single tweet to lowercase.

**Function input:**
- A single string object (tweet) 

**Function output:**
- The tweet in lowercase as a single object

In [ ]:
def to_lower(text):
    # YOUR CODE HERE 
    lower_text = text.lower()
    return lower_text

**Original object:**

In [ ]:
tweet

'RT @RawStory: Researchers say we have three years to act on climate change before it’s too late https://t.co/WdT0KdUr2f https://t.co/Z0ANPT…'

In [ ]:
to_lower(tweet)

'rt @rawstory: researchers say we have three years to act on climate change before it’s too late https://t.co/wdt0kdur2f https://t.co/z0anpt…'

**Expected output:**

```python 
to_lower(tweet)
>> 'rt @rawstory: researchers say we have three years to act on climate change before it’s too late https://t.co/wdt0kdur2f https://t.co/z0anpt…'
```

### Stemming or lemmatising

Write a function that derives the root words for each of the words in a tweet. You may use stemming _or_ lemmatising here.

**Function input:**
- A single string object (tweet) 

**Function output:**
- The tweet in root words as a _single_ object

In [ ]:
def get_roots(text):
    # YOUR CODE HERE 
    stemmer = SnowballStemmer('english')
    root_text = [stemmer.stem(word) for word in text.split()]
    return ' '.join(root_text)

**Original object:**

In [ ]:
tweet

'RT @RawStory: Researchers say we have three years to act on climate change before it’s too late https://t.co/WdT0KdUr2f https://t.co/Z0ANPT…'

In [ ]:
get_roots(tweet)

'rt @rawstory: research say we have three year to act on climat chang befor it too late https://t.co/wdt0kdur2f https://t.co/z0anpt…'

**Expected output:**
```python
get_roots(tweet)
```
Using `SnowballStemmer()`
```python
>> 'rt @rawstory: research say we have three year to act on climat chang befor it too late https://t.co/wdt0kdur2f https://t.co/z0anpt…'
```

Using `WordNetLemmatizer()`
```python
>> 'RT @RawStory: Researchers say we have three years to act on climate change before it’s too late https://t.co/WdT0KdUr2f https://t.co/Z0ANPT…'
```

### Other preprocessing techniques 

Come back here once you have submitted your base model to add any other preprocessing functions yuo wish to add

In [ ]:
# YOUR CODE HERE






### Tokenising 

Write a function that tokenises a single tweet into a list of tokens.

**Function input:**
- A single string object (tweet) 

**Function output:**
- A list of tokens (objects).

In [1]:
def get_tokens(text):
    # YOUR CODE HERE 
    tokens = word_tokenize(text)
    return tokens

**Original object:**

In [2]:
tweet

NameError: ignored

In [ ]:
get_tokens(tweet)

**Expected output:**
```python
get_tokens(tweet)

>> ['RT',
 '@',
 'RawStory',
 ':',
 'Researchers',
 'say',
 'we',
 'have',
 'three',
 'years',
 'to',
 'act',
 'on',
 'climate',
 'change',
 'before',
 'it’s',
 'too',
 'late',
 'https',
 ':',
 '//t.co/WdT0KdUr2f',
 'https',
 ':',
 '//t.co/Z0ANPT…']
```

Besides the preprocessing methods we've already discussed, can you think of how our data can be improved at this stage?

### Vectorising 

Use the code from the webinar to define your vecotiser. For more details on this function have a look at the `sklearn`
<a href="https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html">package documentation on the function</a>. Recall that this function, unlike the others before takes an _**entire column**_ as an argument.

### Putting all the functions together 

Test your cleaning functions on a single tweet. Perform the cleaning procedure on this tweet from start to finish. Make sure the output looks as you would expect. Store the cleaned tweet in `tweet_final`. Don't worry about vectorising for now.

**_Include any other cleaning procedures you add after your base model_**

**Original object:**

In [ ]:
tweet

'RT @RawStory: Researchers say we have three years to act on climate change before it’s too late https://t.co/WdT0KdUr2f https://t.co/Z0ANPT…'

In [ ]:
# clean tweet from start to finish
tweet1 = remove_urls(tweet)
tweet2 = to_lower(tweet1)
tweet3 = get_roots(tweet2)
tweet_final = get_tokens(tweet3)
#.
#.

# cleaned tweet
tweet_final

['rt',
 '@',
 'rawstory',
 ':',
 'research',
 'say',
 'we',
 'have',
 'three',
 'year',
 'to',
 'act',
 'on',
 'climat',
 'chang',
 'befor',
 'it',
 'too',
 'late']

**Expected output:**
```python
tweet_final
>>['rt',
 '@',
 'rawstory',
 ':',
 'research',
 'say',
 'we',
 'have',
 'three',
 'year',
 'to',
 'act',
 'on',
 'climat',
 'chang',
 'befor',
 'it',
 'too',
 'late']
```

### Preprocessing and cleaning entire column of tweets

Now that we know how to clean a single tweet, we can use `apply()` to clean the entire column in the dataframe. This is what the column looks like to begin with.

In [ ]:
train['message'].head()

0    PolySciMajor EPA chief doesn't think carbon di...
1    It's not like we lack evidence of anthropogeni...
2    RT @RawStory: Researchers say we have three ye...
3    #TodayinMaker# WIRED : 2016 was a pivotal year...
4    RT @SoyNovioDeTodas: It's 2016, and a racist, ...
Name: message, dtype: object

Write code that applies your cleaning functions to the entire `message` column of the dataframe using `.apply()`. Do not vectorise just yet. Store this in a new column, `train['message_clean']`. 

In [ ]:
# applying cleaning functions to entire column of dataframe (NO VECTORISING)

# YOUR CODE HERE - use apply with all your cleaning functions 

train['message_clean'] = train['message'].apply(get_roots)
train['message_clean'] = train['message'].apply(get_tokens)
train['message_clean'] = train['message'].apply(remove_urls)
train['message_clean'] = train['message'].apply(to_lower)
train['message_clean'] = train['message'].apply(get_roots)
train['message_clean'] = train['message'].apply(get_tokens)
#train['message_clean']= train['message'].apply(lambda x: (', '.join([x for x in [remove_urls, to_lower, get_roots, get_tokens] if x != None])))
train['message_clean'].head()


0    [PolySciMajor, EPA, chief, does, n't, think, c...
1    [It, 's, not, like, we, lack, evidence, of, an...
2    [RT, @, RawStory, :, Researchers, say, we, hav...
3    [#, TodayinMaker, #, WIRED, :, 2016, was, a, p...
4    [RT, @, SoyNovioDeTodas, :, It, 's, 2016, ,, a...
Name: message_clean, dtype: object

**Expected output:**

```python
train['message_clean'].head()

>>  0    [polyscimajor, epa, chief, does, n't, think, c...
    1    [it, not, like, we, lack, evid, of, anthropoge...
    2    [rt, @, rawstory, :, research, say, we, have, ...
    3    [#, todayinmaker, #, wire, :, 2016, was, a, pi...
    4    [rt, @, soynoviodetodas, :, it, 2016, ,, and, ...
Name: message_clean, dtype: object
```

In [ ]:
def tweet_prep(text):
    text = text.replace('\n', ' ')
    text = re.sub(r"\bhttps://t.co/\w+", '', text)
    text = re.sub('\w*\d\w*', ' ', text) 
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ',text.lower()) 
    
    return text

### Vectorising 

Vectorise your cleaned data using the vectoriser you defined earlier. Don't forget to fit the vectoriser to the data you just cleaned. Store your vectorised data in `train_vec`.

In [ ]:
y = train['sentiment']
X = train['message']

In [487]:
# vectorise your cleaned data

# YOUR CODE HERE 
vect = CountVectorizer(ngram_range=(1,3),
                           max_df = 0.7, min_df = 3, 
                           preprocessor=tweet_prep, 
                           tokenizer=get_tokens, 
                           stop_words='english')
X_vectorized = vect.fit_transform(train['message'])

## Fitting a simple model on training data and evaluate its performance

Fit a model on your cleaned data. Use `train_vec` as your features. Note that you need to convert `train_vec` to an array. 

You are familiar with this process, so there is less guidance here. You may also refer to train notebooks and the webinar pdf as guidance.

Your basic model should be a logistic regression, however if you try different models you will also perform the following procedure.

1. Split the training data into features and labels.
2. Split the training data further into training and validation data.
3. Fit the model on the training subset. 
4. Predict on the validation subset.
5. Calculate the performance metrics on the validation predictions.
6. Select a model based on validation performance (when you have more than one model).
8. Clean the test data.
9. Predict on the cleaned test data.
7. Write a csv that matches `sample_submission.csv`.
8. Submit to Kaggle.

In [488]:
# YOUR CODE HERE 
X_train,X_val,y_train,y_val = train_test_split(X_vectorized,y,test_size=.2,shuffle=True, random_state=12)




In [489]:
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
rfc_pred = rfc.predict(X_val)

In [490]:
f1_score(y_val, rfc_pred, average="macro")

0.5865138394380691

In [491]:
lc = LogisticRegression(C=1.05, penalty='l2', solver='sag', verbose=1)
lc.fit(X_train, y_train)
lc_pred = lc.predict(X_val)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


max_iter reached after 1 seconds


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.3s finished


In [492]:
f1_score(y_val, lc_pred, average="macro")

0.6548100382800901

In [ ]:
nnc = KNeighborsClassifier(3)
nnc.fit(X_train, y_train)
nnc_pred = nnc.predict(X_val)

In [ ]:
f1_score(y_val, nnc_pred, average="macro")

0.4201829665219592

In [ ]:
LinearSVMC = SVC(kernel="linear", C=0.095)
LinearSVMC.fit(X_train, y_train)
LinearSVMC_pred = LinearSVMC.predict(X_val)

In [ ]:
f1_score(y_val, LinearSVMC_pred, average="macro")

0.616539792770931

In [ ]:
dtc = DecisionTreeClassifier(max_depth=9)
dtc.fit(X_train, y_train)
dtc_pred = dtc.predict(X_val)

In [ ]:
f1_score(y_val, dtc_pred, average="macro")

0.33352437755156267

In [ ]:
nbc = MultinomialNB()
nbc.fit(X_train, y_train)
nbc_pred = nbc.predict(X_val)

In [ ]:
f1_score(y_val, nbc_pred, average="macro")

0.6129523633613259

In [ ]:
rbfsvmc = SVC(gamma=2, C=1)
rbfsvmc.fit(X_train, y_train)
rbfsvmc_pred = rbfsvmc.predict(X_val)

In [ ]:
f1_score(y_val, rbfsvmc_pred, average="macro")

0.32433003894249485

In [ ]:
xgbc = XGBClassifier(max_depth=6)
xgbc.fit(X_train, y_train)
xgbc_pred = xgbc.predict(X_val)

In [ ]:
f1_score(y_val, xgbc_pred, average="macro")

0.5147134960630859

In [ ]:
adc = AdaBoostClassifier()
adc.fit(X_train, y_train)
adc_pred = adc.predict(X_val)

In [ ]:
f1_score(y_val, adc_pred, average="macro")

0.4624551342839262

## Suggestions on how to improve the model - Extras 

Here are some ideas on how you can improve your model. Try some of them out once you have submitted your base model to Kaggle. Only use the methods you find improve your model performance.

 - Additional preprocessing 
     - Use some of the methods in the notebooks to balance the data
     - Replace links with the text `Link` to show the machine that there is a link present without the extraneous detail of the link text
     - Remove punctuation
     - Correct spelling
     - Bag of words vs. N-grams - which is better?
     - Try `TweetTokenizer()` - how does this differ from the tokenisers you are familiar with?
 - Consider which other models could do a better job (refer to course resources)
 - Consider some further feature engineering:
     - What other attributes of the tweets may be useful?
     - Should we perform feature selection given their size and sparseness of the vectorised tweets?
 - If you've mastered the above, try implementing hyperparameter tuning methods.
 - Compare all the models you try and remember to submit **the best one** based on test performance (f1-score using macro averaging). 
 
You may use the space below to try out these "Extras".

In [ ]:
# Extras
# YOUR CODE HERE 




## How/why is the model bad?

Do you have any hypotheses as to how you can improve the model further? Use this space to investigate with mindful EDA if necessary.

In [ ]:
# EDA
# YOUR CODE HERE 





Be sure to check whether each additional technique you apply actually improves your model performance.

## Submit your best model predictions 

Don't forget to submit your the predictions on the best model to Kaggle.

In [426]:
testx = test['message']
test_vect = vect.transform(testx)

In [493]:
y_pred = lc.predict(test_vect)

ValueError: ignored

In [429]:
test['sentiment'] = y_pred

NameError: ignored

In [430]:
test.head()

,message,tweetid
0,Europe will now be looking to China to make su...,169760
1,Combine this with the polling of staffers re c...,35326
2,"The scary, unimpeachable evidence that climate...",224985
3,@Karoli @morgfair @OsborneInk @dailykos \nPuti...,476263
4,RT @FakeWillMoore: 'Female orgasms cause globa...,872928


In [ ]:
# write csv for best predictions


